# Catalyst OER Efficiency Optimization with Honegumi

This notebook demonstrates how to use Honegumi to optimize Oxygen Evolution Reaction (OER) catalyst composition, operating conditions, and system parameters to maximize efficiency, durability, and minimize cost.

## Setup and Configuration

First, let's import the necessary libraries and configure Honegumi for our catalyst optimization task.

In [ ]:
from pprint import pprint
from honegumi.core._honegumi import Honegumi
from honegumi.ax.utils import constants as cst
from honegumi.ax._ax import option_rows
import honegumi
import os

# Set up template paths
script_template_dir = honegumi.ax.__path__[0]
core_template_dir = honegumi.core.__path__[0]
script_template_name = "main.py.jinja"
core_template_name = "honegumi.html.jinja"

## Initialize Honegumi

Now we initialize the Honegumi optimizer with our templates.

In [ ]:
# Initialize Honegumi
hg = Honegumi(
    cst,
    option_rows,
    script_template_dir=script_template_dir,
    core_template_dir=core_template_dir,
    script_template_name=script_template_name,
    core_template_name=core_template_name,
)

## Configure for OER Catalyst Optimization

Based on the SDL1 use case requirements, we configure Honegumi with the following options:

1. **Multi-objective**: To optimize for OER efficiency, catalyst durability, and cost simultaneously
2. **Categorical parameters**: To handle electrolyte types (KOH, NaOH, H₂SO₄)
3. **Sum constraint + Composition constraint**: To ensure metal fractions sum to 1
4. **Custom threshold**: To enforce minimum performance criteria (e.g., overpotential < 180 mV)
5. **Single synchrony**: To process one catalyst composition at a time

In [ ]:
# Configure for OER catalyst optimization
options_model = hg.OptionsModel(
    objective="multi",               # For multiple KPIs (efficiency, durability, cost)
    model="Default",                 # Standard GP is suitable for this problem
    task="Single",                   # Single task optimization
    categorical=True,                # For electrolyte type parameter
    sum_constraint=True,             # For metal fractions summing to 1
    order_constraint=False,          # No order constraints specified
    linear_constraint=False,         # No linear constraints beyond composition
    composition_constraint=True,     # Metal fractions must sum to 1
    custom_threshold=True,           # For minimum performance criteria
    existing_data=False,             # No historical data available
    synchrony="Single",              # Process one catalyst composition at a time
    visualize=True                   # Helpful for tracking optimization progress
)

# Print the configured options for review
print("Catalyst OER Optimization Configuration:")
pprint(options_model.model_dump())

## Generate Optimization Script

Now we generate the optimization script based on our configuration.

In [ ]:
# Generate the optimization script
result = hg.generate(options_model)

# Save to file
script_name = "oer_catalyst_optimization.py"
with open(script_name, "w") as f:
    f.write(result)

print(f"\nGenerated optimization script saved to {script_name}")

## Customize the OER Problem Definition

Below, we'll implement the specific parameters for our catalyst OER optimization problem.

In [ ]:
import numpy as np
from ax.service.ax_client import AxClient
from ax.utils.measurement.synthetic_functions import branin
from ax.utils.notebook.plotting import init_notebook_plotting, render

# Initialize plotting for visualization
init_notebook_plotting()

# Create a new Ax client
ax_client = AxClient()

# Define the experiment
ax_client.create_experiment(
    name="oer_catalyst_optimization",
    parameters=[
        # Metal composition parameters (fractions)
        {"name": "Ni", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "Fe", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "Co", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "Mn", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "Cr", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "Zn", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "Cu", "type": "range", "bounds": [0.0, 1.0]},
        
        # Operating parameters
        {"name": "current_density", "type": "choice", "values": [10, 50, 100, 200, 500, 1000]},
        {"name": "temperature", "type": "range", "bounds": [20, 80]},
        {"name": "electrolyte_type", "type": "choice", "values": ["KOH", "NaOH", "H2SO4"]},
        {"name": "electrolyte_pH", "type": "range", "bounds": [1, 14]},
        {"name": "sonication", "type": "choice", "values": ["on", "off"]}
    ],
    objectives={
        "oer_efficiency": "maximize",  # Efficiency (inverse of overpotential)
        "durability": "maximize",     # Hours without performance degradation
        "cost": "minimize"            # Cost of catalyst materials
    },
    parameter_constraints=[
        # Constraint: Metal fractions must sum to 1
        "Ni + Fe + Co + Mn + Cr + Zn + Cu <= 1.0",
        # Temperature constraint for scalability
        "temperature <= 80"
    ],
    outcome_constraints=[
        # Minimum performance criteria
        "oer_efficiency >= -180",  # Overpotential less than 180 mV (negative because it's a cost)
        "durability >= 10"         # At least 10 hours without significant degradation
    ],
)

## Define Evaluation Function

For illustration purposes, we'll use a modified Branin function as a placeholder for your actual experimental evaluation. In a real-world scenario, this would be replaced with your laboratory testing or simulation function.

In [ ]:
def evaluate_catalyst(parameters):
    """Evaluate the catalyst performance using synthetic functions.
    In a real-world application, this function would interface with your
    laboratory equipment or simulation software.
    """
    # Extract parameters
    x1 = parameters.get("Ni", 0) + parameters.get("Fe", 0) * 15  # Scale for branin
    x2 = parameters.get("Co", 0) + parameters.get("Mn", 0) * 15  # Scale for branin
    
    # Use branin as a synthetic test function
    # In reality, this would be your experimental measurement
    base_performance = branin(x1, x2)
    
    # Adjust for additional parameters
    temperature_factor = 1 - (parameters.get("temperature", 20) - 20) / 100  # Higher temp = better efficiency
    
    # Electrolyte type affects performance
    electrolyte_factor = {
        "KOH": 1.0,   # Baseline
        "NaOH": 0.9,  # 10% less efficient
        "H2SO4": 1.2  # 20% more efficient for this simulation
    }.get(parameters.get("electrolyte_type", "KOH"), 1.0)
    
    # Current density affects efficiency
    current_factor = 1 + np.log10(parameters.get("current_density", 10) / 10) * 0.1
    
    # Calculate the three objectives
    oer_efficiency = -base_performance * temperature_factor * electrolyte_factor * current_factor  
    # Negative because lower overpotential is better, but we want to maximize
    
    # Durability calculation - inverse relationship with efficiency
    # More efficient catalysts tend to be less durable in this simulation
    durability = 20 - abs(oer_efficiency) * 0.05
    
    # Cost calculation based on metal composition
    # Costs are arbitrary for this example (cost per unit)
    metal_costs = {
        "Ni": 10,
        "Fe": 5,
        "Co": 30,
        "Mn": 8,
        "Cr": 15,
        "Zn": 7,
        "Cu": 12
    }
    
    cost = sum(parameters.get(metal, 0) * cost_value for metal, cost_value in metal_costs.items())
    
    return {
        "oer_efficiency": oer_efficiency,  # Higher is better (negative overpotential)
        "durability": durability,          # Hours of stable operation
        "cost": cost                       # Lower is better
    }

## Run Optimization

Now let's run the optimization process. In your real-world application, you would perform the actual experiments for each suggested catalyst composition.

In [ ]:
# Number of optimization iterations
total_trials = 13  # As specified in your SDL1 document

for i in range(total_trials):
    # Get the next parameters to try
    parameters, trial_index = ax_client.get_next_trial()
    
    # Normalize metal fractions to ensure they sum to 1
    metals = ["Ni", "Fe", "Co", "Mn", "Cr", "Zn", "Cu"]
    metal_sum = sum(parameters.get(metal, 0) for metal in metals)
    
    if metal_sum > 0:  # Avoid division by zero
        for metal in metals:
            if metal in parameters:
                parameters[metal] = parameters[metal] / metal_sum
    
    # Evaluate the catalyst (in a real scenario, this would be your experiment)
    results = evaluate_catalyst(parameters)
    
    # Log the results
    ax_client.complete_trial(trial_index=trial_index, raw_data=results)
    
    print(f"Trial {i+1}/{total_trials} completed")
    print(f"Parameters: {parameters}")
    print(f"Results: {results}\n")

## Analyze Results

After optimization, we can examine the best catalyst compositions and their performance.

In [ ]:
# Get best parameters and values
best_parameters, values = ax_client.get_best_parameters()

print("Best catalyst composition:")
for param, value in best_parameters.items():
    print(f"{param}: {value}")

print("\nPredicted performance:")
for metric, value in values.items():
    print(f"{metric}: {value}")

## Visualization

Finally, let's visualize the optimization process.

In [ ]:
# Plot the optimization trace
render(ax_client.get_optimization_trace())

# For multi-objective optimization, visualize the Pareto frontier
from ax.service.utils.report_utils import exp_to_df
import plotly.graph_objects as go

# Extract experiment data
exp_df = exp_to_df(ax_client.experiment)

# Create a 3D scatter plot for the three objectives
fig = go.Figure(data=go.Scatter3d(
    x=exp_df["oer_efficiency"],
    y=exp_df["durability"],
    z=exp_df["cost"],
    mode='markers',
    marker=dict(
        size=8,
        color=exp_df.index,  # Color by trial index
        colorscale='Viridis',
        opacity=0.8
    )
))

fig.update_layout(
    title='OER Catalyst Optimization - 3D Pareto Front',
    scene=dict(
        xaxis_title='OER Efficiency (higher is better)',
        yaxis_title='Durability (hours)',
        zaxis_title='Cost (lower is better)'
    ),
    width=800,
    height=800
)

fig.show()

## Conclusion

This notebook demonstrates how to use Honegumi for catalyst OER efficiency optimization with the following characteristics:

1. **Multi-objective optimization**: Balancing efficiency, durability, and cost
2. **Compositional constraint**: Ensuring metal fractions sum to 1
3. **Categorical parameters**: Handling different electrolyte types
4. **Custom thresholds**: Enforcing minimum performance criteria

In a real-world application, you would replace the synthetic evaluation function with your actual experimental protocol.